In [48]:
import torch
import tensorflow as tf
import torch.nn as nn
from transformers import TFAutoModel, AutoModel, AutoTokenizer, GPT2Tokenizer, GPT2Model
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, f1_score
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [36]:
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [38]:
# load the dataset
oba_cleanall = np.load('./TrainData/tweets_oba.npy', allow_pickle=True)
targets_oba = np.load('./TrainData/targets_oba.npy', allow_pickle=True)
rom_cleanall = np.load('./TrainData/tweets_rom.npy', allow_pickle=True)
targets_rom = np.load('./TrainData/targets_rom.npy', allow_pickle=True)
df_oba = pd.DataFrame({'label': targets_oba, 'text':oba_cleanall})
df_rom = pd.DataFrame({'label': targets_rom,'text':rom_cleanall})
print(df_oba.head())
print(df_rom.head())
df_all = df_oba.append(df_rom)
df_all = df_all.sample(frac=1)
df_all.head()
print(df_all.head())

  label                                               text
0     0  Kirkpatrick, who wore a baseball cap embroider...
1     1  #obama debates that Cracker Ass Cracker tonigh...
2     0  @Hollivan @hereistheanswer  Youre missing the ...
3    -1  I was raised as a Democrat  left the party yea...
4     0  The Obama camp can't afford to lower expectati...
  label                                               text
0    -1  Insidious!Mitt Romney's Bain Helped Philip Mor...
1    -1  .@WardBrenda @shortwave8669 @allanbourdius you...
2    -1  Mitt Romney still doesn't believe that we have...
3    -1  Romney's tax plan deserves a 2nd look because ...
4     1  Hope Romney debate prepped w/ the same people ...
     label                                               text
333     -1  I don't care what you think, Romney wants the ...
3779     1                      I'd say obama won this debate
5610     0   #emptybinder Obama's Business Experience     ...
1237    -1  @AC360 @andersoncooper you reall

In [39]:
train, test = train_test_split(df_oba, test_size=0.2, random_state=42)
train, test

(     label                                               text
 184      0  Is "Paid for by Obama Victory Fund" supposed t...
 2728     1   @Semkae I guess u didn't read the links I sen...
 2024     0  MOVIE IDEA: Obama accidentally signs the 'Affo...
 4412    -1                  Michelle Obama is one tall bitch!
 533     -1   Ohio coal miners to Obama: Stop lying about o...
 ...    ...                                                ...
 3772     0   Pres. obama believes private enterprise creat...
 5191     1   Poll shows Obama leading by three http://t.co...
 5226     0   @CarlyRoseMusic Height of popularity!!!!Even ...
 5390     1   Obama you got this! #obama2012 @BarackObama  ...
 860     -1   LIST OF Obama LIES 2012 - UPDATED ... http://...
 
 [4499 rows x 2 columns],
      label                                               text
 3494    -1   Coal miners ask Obama to stop â€˜absolute lie...
 1609     1  #AboutDamnTime RT @rweingarten: Obama was awes...
 1097    -1  FACT: As Presi

In [55]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
# class SimpleGPT2SequenceClassifier(nn.Module):
#     def __init__(
#         self,
#         hidden_size: int,
#         num_classes:int ,
#         max_seq_len:int,
#         gpt_model_name:str,
#     ):
#         super(SimpleGPT2SequenceClassifier,self).__init__()
#         self.gpt2model = GPT2Model.from_pretrained(
#             gpt_model_name
#         )
#         self.fc1 = nn.Linear(hidden_size, num_classes)
#
#     def forward(self, x_in):
#
#         gpt_out = self.gpt2model(x_in)[0] #returns tuple
#         batch_size = gpt_out.shape[0]
#         prediction_vector = self.fc1(gpt_out.view(batch_size,-1)) #(batch_size , max_len, num_classes)
#
#         return prediction_vector

In [41]:
def make_ids(df, label_list, MAX_SEQ_LENGTH):
    input_ids = []
    for t in df.text.values:
        encoded_sent = tokenizer.encode(t,add_special_tokens = True)
        input_ids.append(encoded_sent)
    input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=MAX_SEQ_LENGTH, dtype="long",
                          value=0, truncating="post", padding="post")
    print(input_ids)
    attention_masks = []
    for t in input_ids:
        att_mask = [int(token_id > 0) for token_id in t]
        attention_masks.append(att_mask)

    labels = []
    for l in df.label.values:
        labels.append(label_list.index(l))

    return TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels))

print('Original: ', train.text.values[0])
print(df_oba["label"].unique().tolist().index("-1"))
print('Token IDs:', make_ids(df_oba, df_oba["label"].unique().tolist(), 140))

Original:  Is "Paid for by Obama Victory Fund" supposed to send a positive message?
2
[[    0 25451 26145 ...     0     0     0]
 [    0    85 15044 ...     0     0     0]
 [    0     5     5 ...     0     0     0]
 ...
 [    0    47  9766 ...     0     0     0]
 [    0  1071  8162 ...     0     0     0]
 [    0  5053    31 ...     0     0     0]]
Token IDs: <torch.utils.data.dataset.TensorDataset object at 0x0000017A119A0490>


In [50]:
label_list = train["label"].unique().tolist()
MAX_SEQ_LENGTH=30
batch_size = 16
train_data = make_ids(train, label_list, MAX_SEQ_LENGTH)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = make_ids(test, label_list, MAX_SEQ_LENGTH)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "gpt2", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

[[ 3792   366    47 ...     0     0     0]
 [ 2488 13900    74 ...   531  2486  1839]
 [   44  8874 10008 ...     0     0     0]
 ...
 [ 2488  9914   306 ...  1929    82    24]
 [ 2486   345  1392 ...   220   220   220]
 [39498  3963  2486 ...  4825  2394  1303]]
[[12896 18295  1265 ...    78    11 46669]
 [    2  8585 43343 ...     0     0     0]
 [   37 10659    25 ...   585    19  5167]
 ...
 [ 1303 11275   378 ...     0     0     0]
 [ 8225 42580 13718 ...    85    53    24]
 [ 3137  2497   262 ...     0     0     0]]




Some weights of the model checkpoint at gpt2 were not used when initializing BertForSequenceClassification: ['wte.weight', 'wpe.weight', 'h.0.ln_1.weight', 'h.0.ln_1.bias', 'h.0.attn.bias', 'h.0.attn.c_attn.weight', 'h.0.attn.c_attn.bias', 'h.0.attn.c_proj.weight', 'h.0.attn.c_proj.bias', 'h.0.ln_2.weight', 'h.0.ln_2.bias', 'h.0.mlp.c_fc.weight', 'h.0.mlp.c_fc.bias', 'h.0.mlp.c_proj.weight', 'h.0.mlp.c_proj.bias', 'h.1.ln_1.weight', 'h.1.ln_1.bias', 'h.1.attn.bias', 'h.1.attn.c_attn.weight', 'h.1.attn.c_attn.bias', 'h.1.attn.c_proj.weight', 'h.1.attn.c_proj.bias', 'h.1.ln_2.weight', 'h.1.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.1.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.1.mlp.c_proj.bias', 'h.2.ln_1.weight', 'h.2.ln_1.bias', 'h.2.attn.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.2.attn.c_proj.weight', 'h.2.attn.c_proj.bias', 'h.2.ln_2.weight', 'h.2.ln_2.bias', 'h.2.mlp.c_fc.weight', 'h.2.mlp.c_fc.bias', 'h.2.mlp.c_proj.weight', 'h.2.mlp.c_proj.bias', 'h.3.ln_1.weight', '

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50257, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [51]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50257, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [52]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 10
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [53]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [54]:
import random
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].type(torch.LongTensor).to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # b_input_ids = torch.tensor(b_input_ids).to(torch.int64)
            b_input_ids = b_input_ids.type(torch.LongTensor).to(device)
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
  Batch    40  of    282.    Elapsed: 0:00:06.
  Batch    80  of    282.    Elapsed: 0:00:13.
  Batch   120  of    282.    Elapsed: 0:00:19.
  Batch   160  of    282.    Elapsed: 0:00:25.
  Batch   200  of    282.    Elapsed: 0:00:31.
  Batch   240  of    282.    Elapsed: 0:00:38.
  Batch   280  of    282.    Elapsed: 0:00:44.

  Average training loss: 1.08
  Training epcoh took: 0:00:44

Running Validation...
  Accuracy: 0.43
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...
  Batch    40  of    282.    Elapsed: 0:00:06.
  Batch    80  of    282.    Elapsed: 0:00:13.
  Batch   120  of    282.    Elapsed: 0:00:19.
  Batch   160  of    282.    Elapsed: 0:00:25.
  Batch   200  of    282.    Elapsed: 0:00:31.
  Batch   240  of    282.    Elapsed: 0:00:38.
  Batch   280  of    282.    Elapsed: 0:00:44.

  Average training loss: 0.99
  Training epcoh took: 0:00:44

Running Validation...
  Accuracy: 0.47
  Validation took: 0:00

In [56]:
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()